In [1]:
import pandas as pd
import numpy as np
import networkx as nx

import gurobipy as gp
from gurobipy import GRB

import re
import warnings
warnings.filterwarnings("ignore")

# Import shared calculation functions from module
from calculations import (
    load_location_rankings,
    load_ppu_data,
    categorize_ppus,
    create_ppu_quantities,
    create_renewable_ppu_tracking,
    calculate_max_capacity,
    create_storage_tracking,
    create_incidence_tracking,
    load_cost_data,
    get_component_data,
    calculate_chain_efficiency,
    calculate_chain_cost,
    calculate_ppu_metrics,
    enrich_ppu_quantities
)


# Optimal PPU Mix for Sovereign CO₂-Neutral Energy in Switzerland

## Problem Overview
This optimization problem determines the optimal mix of Power Production Units (PPUs) to achieve Switzerland's energy sovereignty with net-zero CO₂ emissions over an 80-year horizon. The model balances cost minimization, renewable energy maximization, and reduced external energy dependencies through a portfolio-style objective that penalizes expensive, carbon-intensive, and sovereignty-risk energy sources.

---

## Sets and Indices
- **T**: Set of time slices (e.g., 15-minute intervals over a year), indexed by $t \in T$.
- **K**: Set of PPU types (technologies), indexed by $k \in K$.
- **M**: Set of storage types, indexed by $m \in M$.

---

## Parameters
- **Demand and Costs**:
  - $D_t$: Electricity demand in time slice $t$ [kWh].
  - $p_{t,k}$: Delivered cost proxy for PPU $k$ in slice $t$ [CHF/kWh] (e.g., LCOE + grid adder).
  - $e_{t,k}$: Emissions factor for PPU $k$ in slice $t$ [kgCO₂e/kWh].
  - $s_{t,k}$: Sovereignty penalty for PPU $k$ in slice $t$ (dimensionless; larger values indicate less sovereign/more import-reliant energy).
- **Weights and Safeguards**:
  - $\alpha, \beta, \gamma \ge 0$: User-chosen weights for price, emissions, and sovereignty penalty.
  - $\varepsilon > 0$: Small numerical safeguard (e.g., $10^{-6}$ times the median cost) to avoid division by zero.
- **Storage Capacities**:
  - $\text{value}_m$: Storage capacity per GW-PPU for storage type $m$ [MWh/GW-PPU].
  - $k(m)$: The PPU type $k$ that provides storage $m$ (assuming a one-to-one mapping for simplicity; e.g., Lake storage provided by HYD\_S PPU).

---

## Decision Variables
- $x_k \ge 0$: Number of GW of PPU type $k$ to deploy (can be integer or continuous depending on implementation).
- $V_{t,k} \ge 0$: Energy delivered by PPU $k$ during time slice $t$ [kWh].
- $S_{t,m} \ge 0$: Storage level of storage type $m$ at the end of time slice $t$ [MWh] (optional: if storage dynamics are modeled explicitly).

---

## Problem Statement — Aggregate Period Cost and Return (with Demand Balance)
This section defines a portfolio-style objective that maximizes return slice-by-slice over time while meeting demand at every instant. It treats each technology as an asset and penalizes expensive, carbon-intensive, and sovereignty-risk energy.

### Definitions (time- and tech-indexed)
Let $T$ be the set of time slices (e.g., 15-minute intervals), indexed by $t$.  
Let $K$ be the set of technologies / PPUs, indexed by $k$.

- $V_{t,k}$ [kWh]: energy delivered by technology $k$ during slice $t$.
- $p_{t,k}$ [CHF/kWh]: delivered cost proxy (e.g., LCOE + grid adder).
- $e_{t,k}$ [kgCO$_2$e/kWh]: emissions factor.
- $s_{t,k}$ (dimensionless): sovereignty penalty per kWh (larger = less sovereign / more import reliance / lower firmness, etc.).
- $\alpha,\,\beta,\,\gamma \ge 0$: user-chosen weights for price, emissions, and sovereignty penalty.
- $\varepsilon > 0$: small numerical safeguard (e.g., $10^{-6}$ times the median cost) to avoid division by zero.
- $D_t$ [kWh]: electricity demand in slice $t$.

### Composite period cost
We define the composite cost in each slice as a weighted sum of price, emissions, and sovereignty penalty:

$$
\mathrm{cost}_t \;=\; \alpha \sum_{k\in K} p_{t,k} V_{t,k}
\;+\; \beta \sum_{k\in K} e_{t,k} V_{t,k}
\;+\; \gamma \sum_{k\in K} s_{t,k} V_{t,k}.
$$

- First term: monetary expenditure.  
- Second term: environmental externality (can be constrained or priced via $\beta$).  
- Third term: exposure to foreign / unsovereign energy (penalized with $\gamma$).

### Period return (portfolio-style)
Define the return of a slice as the inverse of its composite cost:

$$
\mathrm{Return}_t \;=\; \frac{1}{\mathrm{cost}_t + \varepsilon} \, .
$$

Maximizing $\sum_t \frac{1}{\mathrm{cost}_t + \varepsilon}$ emphasizes the harmonic mean of costs: it penalizes spikes in expensive periods more than a simple arithmetic average would. This matches the risk preference of a power system planner who wants to avoid exposure to high-price scarcity hours. The objective thus pushes the portfolio toward technologies and schedules that keep every slice affordable, not just the average.

### Objective — maximize total return over the year
$$
\max \; \sum_{t\in T} \frac{1}{\mathrm{cost}_t + \varepsilon} \, .
$$

This is equivalent in spirit to “minimize per-slice costs,” but with added downside protection against high-cost intervals.

### Demand balance (must hold every instant)
Electricity must match demand in every time slice:

$$
\sum_{k\in K} V_{t,k} \;=\; D_t \quad \forall\, t\in T \, .
$$

(The above definitions and equations can be used directly when translating the problem into a mathematical programming model or into code for numerical optimization.)

---

## Constraints (Capacity, Storage, Resource, and System Targets)

### 1. Demand Balance (restated)
$$
\sum_{k \in K} V_{t,k} \;=\; D_t \quad \forall t \in T
$$

### 2. PPU Capacity Limits
The energy delivered by each PPU cannot exceed its installed capacity (assuming linear scaling with $x_k$):

$$
V_{t,k} \;\le\; \text{capacity}_{k} \cdot x_k \cdot \Delta t \quad \forall t \in T, \; k \in K
$$

where $\text{capacity}_k$ is the power capacity per GW of PPU $k$ [GW], and $\Delta t$ is the time slice duration (e.g., 0.25 hours for 15-min slices).

### 3. Storage Capacity Limits
The storage levels must not exceed the total available capacity, which scales with the deployed PPUs. The upper limit for each storage type $m$ is:

$$
S_{\max,m} \;=\; x_{k(m)} \cdot \text{value}_m \quad \forall m \in M
$$

If storage dynamics are modeled explicitly, add:

$$
0 \;\le\; S_{t,m} \;\le\; S_{\max,m} \quad \forall t \in T, \; m \in M
$$

with storage balance equations (e.g., inflows from PPUs, outflows to demand, subject to efficiencies). A general update form is:

$$
\Delta S_t \;=\; \sum \bigl(\mathrm{Inflows}_t \cdot \eta_{\mathrm{in}}\bigr) \;-\; \sum \bigl(\mathrm{Outflows}_t / \eta_{\mathrm{out}}\bigr)
$$

### 4. Renewable Resource Availability
For renewable PPUs (e.g., PV, Wind), $V_{t,k}$ is limited by incidence data:

$$
V_{t,k} \;\le\; \text{incidence}_{t,k} \cdot x_k \cdot \Delta t \quad \forall t \in T, \; k \in K_{\text{renewable}}
$$

where $\text{incidence}_{t,k}$ is the available resource (e.g., solar irradiance, wind speed) for PPU $k$ at time $t$.

### 5. Total Energy Target
The total annual energy delivered must meet Switzerland's demand target (e.g., 113 TWh/year):

$$
\sum_{t \in T} \sum_{k \in K} V_{t,k} \;\ge\; 113 \times 10^9 \quad [\text{kWh/year}]
$$

### 6. PPU Deployment Constraint
Each selected PPU type must be deployed at exactly 1 GW:

$$
x_k \;=\; 1 \quad \forall k \in K
$$

This constraint fixes the scale of each PPU unit to 1 GW, simplifying the optimization by treating PPU deployment as a binary selection (deploy or not) rather than a continuous sizing problem.

### 7. Additional Constraints (illustrative)
- **Emissions Cap**: Total annual emissions $\le$ threshold (e.g., for net-zero target).  
- **Renewable Share**: Fraction of energy from renewables $\ge$ minimum percentage.  
- **Sovereignty**: Limit on energy from import-reliant sources.  
- **Grid and Infrastructure**: Limits on total installed capacity, ramp rates, etc.

---

## Cost and Energy Governance Formulas

### 1. Energy Conversion and Efficiency Formulas

#### 1.1 Final Available Energy in the Conversion Chain
The energy output $W_n$ after a series of $n$ components, each with efficiency $\eta_i$, is the product of the initial energy $W_1$ and all individual efficiencies. Auxiliary electricity $\sum E_i$ is added to the final electric component.

$$
W_n \;=\; W_1 \cdot \eta_1 \cdot \dots \cdot \eta_n
$$

*Description:* This multiplicative chain captures losses in sequential transformations (e.g., raw energy $\rightarrow$ transformation $\rightarrow$ electrical). It assumes no parallel paths; auxiliary inputs are post-processed.

#### 1.2 Total Chain Efficiency
The overall efficiency $\eta_{tot}$ of the conversion pathway is the ratio of final to initial energy.

$$
\eta_{tot} \;=\; \frac{W_n}{W_1}
$$

*Description:* Simplifies to $\eta_{tot} = \prod_{i=1}^n \eta_i$ from Equation (1). Used to compare pathways (e.g., direct electrification vs. hydrogen intermediation), highlighting losses in storage/reversal processes.

### 2. Cost Modeling: CAPEX and OPEX Amortization

#### 2.1 Annual Payback for Capital Amortization
The constant annual payback $P_b$ amortizes CAPEX over $n$ years at interest $Z$, solving the annuity equation for zero net present value.

$$
P_b \;=\; \text{CAPEX} \cdot \frac{Z \cdot (1 + Z)^n}{(1 + Z)^n - 1}
$$

*Description:* Derived from the geometric series for loan repayment. Assumes constant annual payments; e.g., $Z = 0.02$ (2% interest). This yields the annualized capital recovery factor (CRF).

#### 2.2 Specific Cost per Energy Unit
The levelized cost contribution $C_W$ for a component combines annualized CAPEX ($P_b$) and operational expenditure (OPEX), normalized by annual energy throughput $W_y$.

$$
C_W \;=\; \frac{P_b + \text{OPEX}}{W_y}
$$

*Description:* OPEX includes maintenance/fuel; $W_y$ is in energy units (e.g., MWh/year). For chains, sum $C_W \cdot W_i$ across components to get total LCOE.

#### 2.3 Levelized Cost of Energy (LCOE)
The total delivered energy cost is the weighted sum of component contributions.

$$
\text{LCOE} \;=\; \sum_i C_{W_i} \cdot W_i
$$

*Description:* Aggregates chain-wide costs, enabling optimization of component sizing (e.g., PV area vs. storage volume). The formulation supports PPU scalability to a 1 GW dispatchable output.

---

## Storage Governance in 15-Minute Timesteps

This section outlines how each storage component's state or availability is governed over 15-minute timesteps. For each storage, we provide the governing inflows and outflows, and the components that can extract resources from it.

### Renewable and Natural Inflows (Uncontrollable)

#### Solar \[Incidence Dependent — No Storage Option]
Inflows: Irradiance from incidence curve \[Incidence]. Outflows: To PV panels.  
Extracted by: PV panels.

**Volume Limit:**
$$
S_{\max} \;=\; 0
$$
No storage; system peak power determined by installed PV (e.g., ~20 GW in the scenario).

#### Wind \[Incidence Dependent — No Storage Option]
Inflows: Wind speed from incidence curve \[Incidence]. Outflows: To wind turbines.  
Extracted by: Wind turbines.

**Volume Limit:**
$$
S_{\max} \;=\; 0
$$
No storage; system peak power determined by installed wind (e.g., ~2 GW in the scenario).

#### River \[Incidence Dependent — No Storage Option]
Inflows: River flow from incidence curve \[Incidence]. Outflows: To hydro-turbines.  
Extracted by: Hydro-turbines (run-of-river).

**Volume Limit:**
$$
S_{\max} \;=\; 0
$$
No storage; energy follows annual/seasonal flow (e.g., ~17.8 TWh/y in the scenario).

#### Lake \[Incidence + Control Dependent]
Inflows: Reservoir inflow from incidence curve \[Incidence], pumped electricity. Outflows: To hydro-turbines.  
Extracted by: Hydro-turbines (reservoir).

**Volume Limit:**
$$
S_{\max} \;=\; 977{,}778~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

### Battery and Electrical Storage

#### Battery
Inflows: Charged electricity. Outflows: Discharged electricity to inverters.  
Extracted by: Inverters (for discharge to grid).

**Volume Limit:**
$$
S_{\max} \;=\; 800~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

### Fuel and Chemical Storage

#### Fuel Tank
Inflows: Production from chains. Outflows: To ICE, gas turbines.  
Extracted by: Internal combustion engines (ICE), gas turbines.

**Volume Limit:**
$$
S_{\max} \;=\; 141{,}320~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### H\(_2\) Storage UG 200 bar
Inflows: Imports, electrolysis production. Outflows: To fuel cells, hydrogen turbines.  
Extracted by: Fuel cells, hydrogen turbines.

**Volume Limit:**
$$
S_{\max} \;=\; 500{,}000~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Liquid H\(_2\) Storage
Inflows: Liquefaction. Outflows: Regasification to fuel cells.  
Extracted by: Fuel cells (after regasification).

**Volume Limit:**
$$
S_{\max} \;=\; 66{,}600~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Solar Concentrator Salt (CSP)
Inflows: Solar input. Outflows: Heat extraction to steam turbines.  
Extracted by: Steam turbines (CSP).

**Volume Limit:**
$$
S_{\max} \;=\; 4{,}000~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Bio-oil
Inflows: Pyrolysis production. Outflows: To diesel engines, boilers.  
Extracted by: Diesel engines, boilers.

**Volume Limit:**
$$
S_{\max} \;=\; 21{,}600~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Palm Oil
Inflows: Imports/production. Outflows: To refineries, engines.  
Extracted by: Refineries (for biodiesel), engines.

**Volume Limit:**
$$
S_{\max} \;=\; 10{,}000~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Wood
Inflows: Harvesting/imports. Outflows: To pyrolysis plants, boilers.  
Extracted by: Pyrolysis plants (for bio-oil), boilers.

**Volume Limit:**
$$
S_{\max} \;=\; 10{,}000~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Biogas (50% CH\(_4\))
Inflows: Anaerobic digestion production. Outflows: To gas engines, turbines.  
Extracted by: Gas engines, turbines.

**Volume Limit:**
$$
S_{\max} \;=\; 60~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### CH\(_4\) Storage — 200 bar
Inflows: Sabatier production. Outflows: To gas turbines, engines.  
Extracted by: Gas turbines, engines.

**Volume Limit:**
$$
S_{\max} \;=\; 10{,}400~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Ammonia Storage
Inflows: Haber–Bosch/electrolysis production. Outflows: To ammonia crackers, turbines.  
Extracted by: Ammonia crackers (for H\(_2\)), turbines (direct combustion).

**Volume Limit:**
$$
S_{\max} \;=\; 35{,}360~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

---

## Solution Approach
- **Mathematical Programming**: Solve using solvers like Gurobi or CPLEX, with $V_{t,k}$ as continuous variables and $x_k$ as integer or continuous.
- **Time Series Simulation**: For large $|T|$ (e.g., 35,040 for 15-min slices), use decomposition or rolling horizon methods.
- **Sensitivity Analysis**: Vary $\alpha, \beta, \gamma$ to explore trade-offs between cost, emissions, and sovereignty.

This formulation provides a complete, classroom-ready optimization problem that can be extended with additional details as needed.


In [2]:
# Vector of raw energy sources/storage components with default integer values and typical units
raw_energy_storage = [
    {"storage": "Lake", "value": 977778, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Hydro Turb"]},
    {"storage": "Fuel Tank", "value": 141320, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE", "Gas Turbine"]},
    {"storage": "H2 Storage UG 200bar", "value": 500000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Fuel Cell", "Combined cycle power plant"]},
    {"storage": "Liquid storage", "value": 66600, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Fuel Cell"]},
    {"storage": "Solar concentrator salt", "value": 4000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Solar concentrator steam"]},
    {"storage": "Biooil", "value": 21600, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE", "Gas Turbine"]},
    {"storage": "Palm oil", "value": 10000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE"]},
    {"storage": "Biogas (50% CH4)", "value": 60, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Gas Turbine"]},
    {"storage": "CH4 storage 200bar", "value": 10400, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Gas Turbine"]},
    {"storage": "Ammonia storage", "value": 35360, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Ammonia cracking"]}
]

raw_energy_incidence = [
    {"storage": "Wood", "value": 10000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Pyrolysis"]},
    {"storage": "River", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Hydro Turb"]},
    {"storage": "Solar", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["PV"]},
    {"storage": "Wind", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Wind (onshore)", "Wind (offshore)"]}
]

In [3]:
# ============================================================================
# FULL MODEL SETUP: Functional Composition for Optimization Prep
# ============================================================================

import pandas as pd
import numpy as np
import networkx as nx  # Unused here but imported in original; keep for future
import gurobipy as gp  # For later optimization
from gurobipy import GRB
import re
import warnings
warnings.filterwarnings("ignore")

# Import all from calculations.py (assumes additions above are pasted)
from calculations import (
    # Existing
    load_location_rankings, load_ppu_data, categorize_ppus, create_ppu_quantities,
    create_renewable_ppu_tracking, calculate_max_capacity, create_storage_tracking,
    create_incidence_tracking, load_cost_data, get_component_data,
    calculate_chain_efficiency, calculate_chain_cost, calculate_ppu_metrics,
    enrich_ppu_quantities, get_incidence_data, update_storage, update_incidence,
    # New FP helpers
    pipeline, compose_ppu_setup, compose_storage_tracking, compose_incidence_tracking,
    compose_renewable_tracking, update_all_storages, add_ppu
)

# ============================================================================
# 1. HYPERPARAMETERS (Immutable Config)
# ============================================================================
def get_hyperparams():
    """Pure function: Return all hyperparameters as a frozen dict for immutability."""
    return {
        'T': np.arange(96 * 365),  # 35,040 timesteps
        'n_timesteps': 96 * 365,
        'delta_t': 0.25,  # hours
        'epsilon': 1e-6,
        'weights': {'alpha': 1.0, 'beta': 0.1, 'gamma': 0.5},
        'annual_demand_target': 113e9,  # kWh/year
        'solar_area_m2_per_gw': 10e6,  # m²
        'wind_turbines_offshore_per_gw': 100,
        'wind_turbines_onshore_per_gw': 300,
        'lake_max': 8.9e7,  # MWh
        'lake_growth': 0.005,
        'quantity_default': 10,
        'capacity_gw_default': 1.0
    }

# Vectors for energy tracking 
raw_energy_storage = [
    {"storage": "Lake", "value": 977778, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Hydro Turb"]},
    {"storage": "Fuel Tank", "value": 141320, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE", "Gas Turbine"]},
    {"storage": "H2 Storage UG 200bar", "value": 500000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Fuel Cell", "Combined cycle power plant"]},
    {"storage": "Liquid storage", "value": 66600, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Fuel Cell"]},
    {"storage": "Solar concentrator salt", "value": 4000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Solar concentrator steam"]},
    {"storage": "Biooil", "value": 21600, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE", "Gas Turbine"]},
    {"storage": "Palm oil", "value": 10000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE"]},
    {"storage": "Biogas (50% CH4)", "value": 60, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Gas Turbine"]},
    {"storage": "CH4 storage 200bar", "value": 10400, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Gas Turbine"]},
    {"storage": "Ammonia storage", "value": 35360, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Ammonia cracking"]}
]

raw_energy_incidence = [
    {"storage": "Wood", "value": 10000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Pyrolysis"]},
    {"storage": "River", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Hydro Turb"]},
    {"storage": "Solar", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["PV"]},
    {"storage": "Wind", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Wind (onshore)", "Wind (offshore)"]}
]

# ============================================================================
# 2. PPU optimization vector
# ============================================================================

# I need a data frame ppu_dictionnary that contains all PPU with their respective information
# It should initially contain the following columns and be empty elsewise - meaning no stored information to begin with:
# - PPU ID
# - PPU type (HYD_S, SOL_SALT, WIND_OFFSHORE, etc.) - here we refer to ppu_constructs_components.csv to get the name and corresponding information
# - PPU category (production, storage, etc.) - here we refer to ppu_constructs_components.csv to get the name and corresponding information
# - PPU chain efficiency - calculated based on the components in the PPU chain using the values in cost_table_tidy.csv
# - PPU chain cost in CHF/kWh - calculated based on the components in the PPU chain using the values in cost_table_tidy.csv
# - PPU chain cost in CHF/kWh per quarter hour so 0.25*(PPU chain cost in CHF/kWh) 
# - If solar or wind provide the location ranking. Else wise set to NaN

# I then need a function that populates this data frame with all the PPU information I just required 
# This function should take as input the empty data frame, the PPU type I wish to add. 
# - it automatically adds the PPU
# - gives it a unique ID (incremental integer starting from 1) 
# - calculates all the required information based on the PPU type and the data in cost_table_tidy.csv and ppu_constructs_components.csv
# - If solar or wind provide the location ranking. Else wise set to NaN. The location ranking data is given by next_available_location(ppu_dictionnary) function

# I then need a storage verification function that checks if all storages are present.
# the function covers the ppu_dictionnary, counts the storages present, and compares the used capacity to the possible capacity given by the raw_energy_storage vector value multiplied by the number of PPU instances.

# If a new PPU is solar or wind I need to look into the ppu_dictionnary to find all the solar/wind ppu and verify the locations they have already taken. Later I need to assign the next available location to the new PPU being added.
# This function should return the next available location ranking for solar and wind PPUs.
# This function should verify that over the entire ppu_dictionnary no two solar or wind PPUs have the same location ranking.



## PPU Dictionary Management System

This section demonstrates the new PPU dictionary management functions implemented in `calculations.py`:

### Core Functions

1. **`initialize_ppu_dictionary()`**
   - Creates an empty DataFrame with all required columns
   - Columns: PPU_ID, PPU_Name, PPU_Category, Chain_Efficiency, Cost_CHF_per_kWh, Cost_CHF_per_Quarter_Hour, Location_Rank, Components

2. **`add_ppu_to_dictionary()`**
   - Adds a new PPU with automatic ID assignment (incremental)
   - Calculates chain efficiency from components
   - Calculates costs from cost_table_tidy.csv
   - Automatically assigns location ranking for solar/wind PPUs
   - Returns updated dictionary

3. **`next_available_location()`**
   - Finds the next best available location for solar/wind PPUs
   - Checks existing assignments to avoid duplicates
   - Returns location details (rank, lat, lon, potential)
   - Validates uniqueness across all renewable PPUs

4. **`verify_storage_capacity()`**
   - Analyzes storage usage across all PPUs
   - Compares used capacity to available capacity
   - Returns detailed report per storage type
   - Shows which PPUs use each storage

5. **`verify_unique_locations()`**
   - Ensures no two PPUs share the same location
   - Detects and reports duplicate assignments
   - Provides summary of all renewable PPU locations

### Usage Pattern

```python
# 1. Initialize
ppu_dictionary = initialize_ppu_dictionary()

# 2. Add PPUs
ppu_dictionary = add_ppu_to_dictionary(ppu_dictionary, 'PV', ...)
ppu_dictionary = add_ppu_to_dictionary(ppu_dictionary, 'WD_OFF', ...)

# 3. Verify
location_report = verify_unique_locations(ppu_dictionary)
storage_report = verify_storage_capacity(ppu_dictionary, raw_energy_storage, ppu_constructs_df)
```

### Data Flow

1. Load PPU constructs from `ppu_constructs_components.csv`
2. Load cost data from `cost_table_tidy.csv`
3. Load location rankings (solar/wind)
4. Add PPUs one by one with automatic calculations
5. Verify integrity (unique locations, storage capacity)

In [4]:
# ============================================================================
# PPU DICTIONARY MANAGEMENT - New Implementation
# ============================================================================

# Import the new PPU dictionary functions
from calculations import (
    initialize_ppu_dictionary,
    add_ppu_to_dictionary,
    next_available_location,
    verify_storage_capacity,
    verify_unique_locations
)

# Initialize hyperparameters if not already defined
if 'hyperparams' not in globals():
    hyperparams = get_hyperparams()

# Load necessary data
ppu_constructs_df = load_ppu_data('data/ppu_constructs_components.csv')
cost_df = load_cost_data('data/cost_table_tidy.csv')
solar_locations_df = load_location_rankings('solar')
wind_locations_df = load_location_rankings('wind')

# ============================================================================
# 1. Initialize Empty PPU Dictionary
# ============================================================================
ppu_dictionary = initialize_ppu_dictionary()
print("Initialized empty PPU dictionary:")
print(ppu_dictionary)
print(f"\nColumns: {list(ppu_dictionary.columns)}")

# ============================================================================
# 2. Add PPUs to Dictionary
# ============================================================================
print("\n" + "="*80)
print("ADDING PPUs TO DICTIONARY")
print("="*80)

# Add a hydroelectric PPU (no location needed)
ppu_dictionary = add_ppu_to_dictionary(
    ppu_dictionary=ppu_dictionary,
    ppu_name='HYD_S',
    ppu_constructs_df=ppu_constructs_df,
    cost_df=cost_df,
    solar_locations_df=solar_locations_df,
    wind_locations_df=wind_locations_df,
    delta_t=hyperparams['delta_t']
)

# Add a solar PPU (location assigned automatically)
ppu_dictionary = add_ppu_to_dictionary(
    ppu_dictionary=ppu_dictionary,
    ppu_name='PV',
    ppu_constructs_df=ppu_constructs_df,
    cost_df=cost_df,
    solar_locations_df=solar_locations_df,
    wind_locations_df=wind_locations_df,
    delta_t=hyperparams['delta_t']
)

# Add another solar PPU (should get next best location)
ppu_dictionary = add_ppu_to_dictionary(
    ppu_dictionary=ppu_dictionary,
    ppu_name='SOL_SALT',
    ppu_constructs_df=ppu_constructs_df,
    cost_df=cost_df,
    solar_locations_df=solar_locations_df,
    wind_locations_df=wind_locations_df,
    delta_t=hyperparams['delta_t']
)

# Add a wind PPU
ppu_dictionary = add_ppu_to_dictionary(
    ppu_dictionary=ppu_dictionary,
    ppu_name='WD_OFF',
    ppu_constructs_df=ppu_constructs_df,
    cost_df=cost_df,
    solar_locations_df=solar_locations_df,
    wind_locations_df=wind_locations_df,
    delta_t=hyperparams['delta_t']
)

# Add a storage PPU
ppu_dictionary = add_ppu_to_dictionary(
    ppu_dictionary=ppu_dictionary,
    ppu_name='H2_G',
    ppu_constructs_df=ppu_constructs_df,
    cost_df=cost_df,
    solar_locations_df=solar_locations_df,
    wind_locations_df=wind_locations_df,
    delta_t=hyperparams['delta_t']
)

print("\n" + "="*80)
print("CURRENT PPU DICTIONARY")
print("="*80)
print(ppu_dictionary)

# ============================================================================
# 3. Verify Unique Locations
# ============================================================================
print("\n" + "="*80)
print("LOCATION UNIQUENESS VERIFICATION")
print("="*80)

location_report = verify_unique_locations(ppu_dictionary)
print(f"Status: {location_report['message']}")
print(f"Total renewable PPUs: {location_report['total_renewable_ppus']}")
print(f"Is unique: {location_report['is_unique']}")
if not location_report['is_unique']:
    print(f"Duplicate ranks: {location_report['duplicate_ranks']}")
    print(f"PPUs by rank: {location_report['ppus_by_rank']}")

# ============================================================================
# 4. Verify Storage Capacity
# ============================================================================
print("\n" + "="*80)
print("STORAGE CAPACITY VERIFICATION")
print("="*80)

storage_report = verify_storage_capacity(
    ppu_dictionary=ppu_dictionary,
    raw_energy_storage=raw_energy_storage,
    ppu_constructs_df=ppu_constructs_df
)

print(f"\nSummary:")
print(f"  Total storage types: {storage_report['summary']['total_storages']}")
print(f"  Storages in use: {storage_report['summary']['storages_in_use']}")
print(f"  All storages OK: {storage_report['summary']['all_storages_ok']}")

print("\nDetailed Storage Report:")
for storage_name, info in storage_report.items():
    if storage_name != 'summary' and info['num_ppu_instances'] > 0:
        print(f"\n  {storage_name}:")
        print(f"    Base capacity: {info['base_capacity_per_gw']:,.0f} {info['unit']}")
        print(f"    PPU instances: {info['num_ppu_instances']}")
        print(f"    Total available: {info['total_available_capacity']:,.0f} {info['unit']}")
        print(f"    Used by PPUs: {', '.join(info['ppu_names'])}")
        print(f"    Status: {info['status']}")

# ============================================================================
# 5. Check Next Available Location
# ============================================================================
print("\n" + "="*80)
print("NEXT AVAILABLE LOCATIONS")
print("="*80)

# Check next available solar location
next_solar = next_available_location(
    ppu_dictionary=ppu_dictionary,
    renewable_type='solar',
    solar_locations_df=solar_locations_df,
    wind_locations_df=wind_locations_df
)
if next_solar:
    print(f"\nNext available solar location:")
    print(f"  Rank: {next_solar['rank']}")
    print(f"  Coordinates: ({next_solar['lat']:.4f}, {next_solar['lon']:.4f})")
    print(f"  Potential: {next_solar['potential']:.2f}")

# Check next available wind location
next_wind = next_available_location(
    ppu_dictionary=ppu_dictionary,
    renewable_type='wind_offshore',
    solar_locations_df=solar_locations_df,
    wind_locations_df=wind_locations_df
)
if next_wind:
    print(f"\nNext available wind location:")
    print(f"  Rank: {next_wind['rank']}")
    print(f"  Coordinates: ({next_wind['lat']:.4f}, {next_wind['lon']:.4f})")
    print(f"  Potential: {next_wind['potential']:.2f}")

print("\n" + "="*80)
print("PPU DICTIONARY SETUP COMPLETE")
print("="*80)

Initialized empty PPU dictionary:
Empty DataFrame
Columns: [PPU_ID, PPU_Name, PPU_Category, Chain_Efficiency, Cost_CHF_per_kWh, Cost_CHF_per_Quarter_Hour, Location_Rank, Components]
Index: []

Columns: ['PPU_ID', 'PPU_Name', 'PPU_Category', 'Chain_Efficiency', 'Cost_CHF_per_kWh', 'Cost_CHF_per_Quarter_Hour', 'Location_Rank', 'Components']

ADDING PPUs TO DICTIONARY
Added PPU 'HYD_S' (ID: 1, Category: Production, Efficiency: 0.8800, Cost: 0.239341 CHF/kWh)
Added PPU 'PV' (ID: 2, Category: Production, Efficiency: 0.8374, Cost: 0.304971 CHF/kWh, Location Rank: 1)
Added PPU 'SOL_SALT' (ID: 3, Category: Production, Efficiency: 0.0250, Cost: 0.786526 CHF/kWh)
Added PPU 'WD_OFF' (ID: 4, Category: Production, Efficiency: 0.8374, Cost: 0.329168 CHF/kWh, Location Rank: 2)
Added PPU 'H2_G' (ID: 5, Category: Storage, Efficiency: 0.0900, Cost: 613.423945 CHF/kWh)

CURRENT PPU DICTIONARY
  PPU_ID  PPU_Name PPU_Category  Chain_Efficiency  Cost_CHF_per_kWh  \
0      1     HYD_S   Production          0

### Data Format Verification

The location ranking files have the following column structure:

**Solar locations:** `latitude`, `longitude`, `mean_solar_incidence_kwh_m2_per_hour`, `rank`

**Wind locations:** `latitude`, `longitude`, `mean_wind_speed_m_per_s`, `rank`

The `load_location_rankings()` function automatically adds a standardized `potential` column for easier access, which maps to the appropriate mean value column.

### Example: Adding Multiple PPU Instances

Below is an example workflow showing how to build up a complete PPU portfolio:

In [5]:
# ============================================================================
# EXAMPLE: Building a Complete PPU Portfolio
# ============================================================================

# Start fresh
ppu_portfolio = initialize_ppu_dictionary()

# Define PPU types to add (mix of production and storage)
ppus_to_add = [
    'HYD_S',      # Hydro with storage
    'HYD_R',      # Run-of-river hydro
    'PV',         # Solar PV (will get location rank 1)
    'WD_ON',      # Onshore wind (will get location rank 1)
    'WD_OFF',     # Offshore wind (will get location rank 1)
    'THERM',      # Thermal power
    'H2_G',       # Hydrogen storage
]

print("Building PPU Portfolio...")
print("="*80)

for ppu_name in ppus_to_add:
    try:
        ppu_portfolio = add_ppu_to_dictionary(
            ppu_dictionary=ppu_portfolio,
            ppu_name=ppu_name,
            ppu_constructs_df=ppu_constructs_df,
            cost_df=cost_df,
            solar_locations_df=solar_locations_df,
            wind_locations_df=wind_locations_df,
            delta_t=hyperparams['delta_t']
        )
    except Exception as e:
        print(f"Error adding {ppu_name}: {e}")

print("\n" + "="*80)
print("FINAL PPU PORTFOLIO")
print("="*80)
print(ppu_portfolio[['PPU_ID', 'PPU_Name', 'PPU_Category', 'Chain_Efficiency', 
                     'Cost_CHF_per_kWh', 'Location_Rank']].to_string(index=False))

# Final verification
print("\n" + "="*80)
print("FINAL VERIFICATION")
print("="*80)

loc_check = verify_unique_locations(ppu_portfolio)
print(f"\n✓ Location Check: {loc_check['message']}")

storage_check = verify_storage_capacity(ppu_portfolio, raw_energy_storage, ppu_constructs_df)
print(f"✓ Storage Check: {storage_check['summary']['storages_in_use']}/{storage_check['summary']['total_storages']} storage types in use")

print("\n" + "="*80)
print("PORTFOLIO COMPLETE!")
print("="*80)

Building PPU Portfolio...
Added PPU 'HYD_S' (ID: 1, Category: Production, Efficiency: 0.8800, Cost: 0.239341 CHF/kWh)
Added PPU 'HYD_R' (ID: 2, Category: Production, Efficiency: 0.8800, Cost: 0.239341 CHF/kWh)
Added PPU 'PV' (ID: 3, Category: Production, Efficiency: 0.8374, Cost: 0.304971 CHF/kWh, Location Rank: 1)
Added PPU 'WD_ON' (ID: 4, Category: Production, Efficiency: 0.8374, Cost: 0.300544 CHF/kWh, Location Rank: 2)
Added PPU 'WD_OFF' (ID: 5, Category: Production, Efficiency: 0.8374, Cost: 0.329168 CHF/kWh, Location Rank: 3)
Added PPU 'THERM' (ID: 6, Category: Production, Efficiency: 0.4214, Cost: 0.233785 CHF/kWh)
Added PPU 'H2_G' (ID: 7, Category: Storage, Efficiency: 0.0900, Cost: 613.423945 CHF/kWh)

FINAL PPU PORTFOLIO
PPU_ID PPU_Name PPU_Category  Chain_Efficiency  Cost_CHF_per_kWh  Location_Rank
     1    HYD_S   Production          0.880000          0.239341            NaN
     2    HYD_R   Production          0.880000          0.239341            NaN
     3       PV   P

# 15-Minute Energy Dispatch State Machine

## Framework for Disposition–Utility–Cost Indices

This framework formalizes the decision logic for determining **which PPUs discharge** during **energy shortfall** and **which PPUs absorb** during **energy surplus**. The system logs per-PPU costs and computes the Herfindahl-Hirschman Index (HHI) by **PPU type** to measure market concentration.

---

## 0) Time Base, Smoothing, and Notation

### Time Discretization
- Discrete time steps: $t=1,\dots,T$, with each step representing $\Delta=15$ minutes
- Total timesteps: $T = 35{,}040$ (96 steps/day × 365 days)

### Exponential Moving Average (EMA)
For any raw time series $x_t$, the smoothed value is:
$$
\overline{x}_t=(1-\beta)\,x_t+\beta\,\overline{x}_{t-1},\qquad \beta=0.2
$$

### Optional Squashing Function
To map values to $[-1,1]$, we use the hyperbolic tangent with scaling parameter $\alpha$:
$$
\sigma_\alpha(x)=\tanh\!\big(x/\alpha\big)
$$
**Note:** Scaling parameters $\alpha_d$, $\alpha_u$, $\alpha_c$ should be calibrated during implementation.

### Entities and Constants
- **Indices:**
  - $i$: Individual PPU instance index
  - $k$: PPU **type** (e.g., Hydro-Storage, Hydro-RoR, Battery, CCGT, Wind, PV)
  
- **Storage Parameters:**
  - $S_i$: Storage energy capacity (MWh) for PPU $i$
  - **Normalized State of Charge (SoC):** $\tilde{s}_{i,t}\in[0,1]$ with:
    - Initial condition: $\tilde{s}_{i,0}=0.60$
    - Target setpoint: $\tilde{s}^{\star}_i=0.60$
    - Deadband: $\delta_i=0.05$
  
- **Efficiencies:**
  - $\eta_i^{\uparrow}$: Charging efficiency
  - $\eta_i^{\downarrow}$: Discharging efficiency
  - **Note:** All operational limits (ramp rates, min/max power) are handled externally

### System-Level Signals (Exogenous Inputs)
- $\mathrm{demand}_t$ (MW): Total electricity demand
- $\mathrm{nonflex\_supply}_t$ (MW): Non-flexible supply (baseload, must-run generation)
- $\pi_t$ (€/MWh): Electricity price or scarcity signal

---

## 1) System Energy Balance (Net Shortfall)

The net system need at each timestep is:
$$
\Phi_t \;=\; D^{\text{net}}_t \;=\; \mathrm{demand}_t \;-\; \mathrm{nonflex\_supply}_t
$$

**Interpretation:**
- $\Phi_t>0$: **Shortfall** → The grid requires additional supply (discharge or flexible production)
- $\Phi_t<0$: **Surplus** → Excess energy available (suitable for charging or curtailment)

We typically use the smoothed value $\overline{\Phi}_t$ (EMA with $\beta=0.2$) to reduce noise and avoid over-reaction to instantaneous fluctuations.

---

## 2) Per-PPU Decision Indices in $[-1,1]$

Each PPU maintains three indices that guide dispatch decisions:

### 2.1 Disposition Index (Storage PPUs Only)

**Question:** *How willing is this storage unit to discharge now?*

The disposition index measures deviation from the target SoC, with a deadband to avoid unnecessary cycling:

$$
\Delta \tilde{s}_{i,t}=
\begin{cases}
\tilde{s}_{i,t} - (\tilde{s}^{\star}_i+\delta_i), & \text{if } \tilde{s}_{i,t} > \tilde{s}^{\star}_i+\delta_i & \text{(above target)}\\[6pt]
0, & \text{if } |\tilde{s}_{i,t}-\tilde{s}^{\star}_i| \le \delta_i & \text{(within deadband)}\\[6pt]
\tilde{s}_{i,t} - (\tilde{s}^{\star}_i-\delta_i), & \text{if } \tilde{s}_{i,t} < \tilde{s}^{\star}_i-\delta_i & \text{(below target)}
\end{cases}
$$

Normalize by the maximum possible excursion beyond the deadband:
$$
\widehat{\Delta}\tilde{s}_{i,t}
=\frac{\Delta \tilde{s}_{i,t}}
{\max\{\,1-\tilde{s}^{\star}_i-\delta_i,\;\tilde{s}^{\star}_i-\delta_i\,\}}
$$

Apply optional squashing to obtain the disposition index:
$$
d^{\text{stor}}_{i,t}=\sigma_{\alpha_d}\!\big(\widehat{\Delta}\tilde{s}_{i,t}\big)\;\in[-1,1]
$$

**Interpretation:**
- $d^{\text{stor}}_{i,t} = +1$: Storage is full → **strongly willing** to discharge
- $d^{\text{stor}}_{i,t} = -1$: Storage is nearly empty → **reluctant** to discharge (prefers to charge)
- $d^{\text{stor}}_{i,t} = 0$: Storage is at target → neutral disposition

### 2.2 Utility Indices (System-Wide Context)

**Question:** *How valuable is it to discharge/charge given current system conditions?*

#### Discharge Utility
Higher during shortfall (when supply is needed):
$$
u^{\text{dis}}_{i,t}=\sigma_{\alpha_u}\!\big(\overline{\Phi}_t\big)\;\in[-1,1]
$$

#### Charge Utility
Non-zero only during surplus (when excess energy is available):
$$
u^{\text{chg}}_{i,t}=\mathbf{1}\{\Phi_t<0\}\,\sigma_{\alpha_u}\!\big(-\overline{\Phi}_t\big)\;\in[-1,1]
$$

**Interpretation:**
- High $u^{\text{dis}}_{i,t}$ during energy shortfall → discharging benefits the grid
- High $u^{\text{chg}}_{i,t}$ during energy surplus → charging absorbs excess energy

### 2.3 Cost Index (Multi-Horizon Co-state)

**Question:** *How does the current price compare to future value of stored energy?*

Let $\lambda^{(H)}_{i,t}$ be the **co-state** (marginal value of stored energy, €/MWh) derived from dynamic programming with look-ahead horizon $H\in\{1\text{d},3\text{d},7\text{d},30\text{d}\}$.

For each horizon:
$$
c^{(H)}_{i,t}=\sigma_{\alpha_c}\!\Big(\overline{\pi_t-\lambda^{(H)}_{i,t}}\Big)
$$

Aggregate across all horizons with equal weights:
$$
c_{i,t}=\tfrac{1}{4}\sum_{H\in\{1\text{d},3\text{d},7\text{d},30\text{d}\}} c^{(H)}_{i,t}\;\in[-1,1]
$$

**Implementation Note:** If dynamic programming is not yet implemented, use the proxy:
$$
\lambda^{(H)}_{i,t}\approx \eta_i^{\downarrow}\,\mathbb{E}_t[\pi_{t+H}]
$$
where $\mathbb{E}_t[\pi_{t+H}]$ is the expected future price at horizon $H$.

**Interpretation:**
- $c_{i,t} > 0$: Current price $\pi_t$ exceeds future value → **favorable** to discharge now
- $c_{i,t} < 0$: Future value exceeds current price → **better to save** energy

---

## 3) Composite Benefit and Cost Metrics

### Discharge Benefit
Combines disposition, utility, and cost with equal weights:
$$
B^{\text{dis}}_{i,t}=\tfrac{1}{3}\big(d^{\text{stor}}_{i,t}+u^{\text{dis}}_{i,t}+c_{i,t}\big)\in[-1,1]
$$

Convert to a non-negative cost (lower is better):
$$
\kappa^{\text{dis}}_{i,t}=1-B^{\text{dis}}_{i,t}\in[0,2]
$$

### Charge Benefit
For charging, we flip the disposition and cost indices:
$$
B^{\text{chg}}_{i,t}=\tfrac{1}{3}\big((-d^{\text{stor}}_{i,t})+u^{\text{chg}}_{i,t}+(-c_{i,t})\big)\in[-1,1]
$$

Convert to a non-negative cost:
$$
\kappa^{\text{chg}}_{i,t}=1-B^{\text{chg}}_{i,t}\in[0,2]
$$

**Key Principle:** Lower $\kappa$ values indicate more attractive dispatch options.

---

## 4) State Machine Logic (per 15-Minute Timestep)

### Energy Balance After Collection

Let:
- $E^{\text{col}}_t$: Energy collected from renewable incidence (MWh)
- $\eta^{\text{col}}_t$: Collection efficiency
- $D^{\text{need}}_t$: Gross demand energy (MWh)

Net energy requirement after accounting for collected energy:
$$
E^{\text{net}}_t \;=\; D^{\text{need}}_t \;-\; \eta^{\text{col}}_t\,E^{\text{col}}_t
$$

---

### Case A — Balanced System $(E^{\text{net}}_t=0)$

**Actions:**
1. Compute and store all indices $(d^{\text{stor}}_{i,t},\,u^{\text{dis}}_{i,t},\,c_{i,t})$ for theoretical evaluation
2. No energy reallocation required
3. Maintain current SoC levels

---

### Case B — Shortfall $(E^{\text{net}}_t>0)$

**Procedure:** `collect_energy_from_PPU($E^{\text{net}}_t$)`

**Step 1:** Calculate dispatch costs
- For all eligible PPUs, compute $(d,u^{\text{dis}},c)$ and the discharge cost $\kappa^{\text{dis}}_{i,t}$

**Step 2:** Compute direct cost shares
$$
C_t=\sum_i \kappa^{\text{dis}}_{i,t},\qquad
w^{\text{dis}}_{i,t}=\frac{\kappa^{\text{dis}}_{i,t}}{C_t}
$$

**Step 3:** Allocate inversely to cost (cheaper PPUs dispatch more)

With small safeguard $\varepsilon>0$ to avoid division by zero:
$$
\tilde{w}^{\text{dis}}_{i,t}=\frac{1/\max(\kappa^{\text{dis}}_{i,t},\varepsilon)}{\sum_j 1/\max(\kappa^{\text{dis}}_{j,t},\varepsilon)}
$$

Target discharge energy for each PPU:
$$
E^{\text{dis,tar}}_{i,t}=\tilde{w}^{\text{dis}}_{i,t}\;E^{\text{net}}_t
$$

**Step 4:** Feasibility check
- Apply operational constraints (SoC limits, ramp rates, minimum power)
- Adjust targets to feasible levels: $E^{\text{dis,act}}_{i,t}$
- Reassign any residual energy to the next lowest-cost PPU

**Step 5:** Update state of charge
$$
\tilde{s}_{i,t+1}=\mathrm{clamp}\!\left(\tilde{s}_{i,t}-\frac{E^{\text{dis,act}}_{i,t}}{S_i}\cdot\frac{1}{\eta_i^{\downarrow}},\,0,\,1\right)
$$

---

### Case C — Surplus $(E^{\text{net}}_t<0)$

**Procedure:** `distribute_energy_to_storage($|E^{\text{net}}_t|$)`

**Step 1:** Calculate charging costs
- For all storage PPUs, compute $(d,u^{\text{chg}},c)$ and the charge cost $\kappa^{\text{chg}}_{i,t}$

**Step 2:** Compute direct cost shares
$$
C'_t=\sum_i \kappa^{\text{chg}}_{i,t},\qquad
w^{\text{chg}}_{i,t}=\frac{\kappa^{\text{chg}}_{i,t}}{C'_t}
$$

**Step 3:** Allocate inversely to cost (cheaper charging gets more energy)
$$
\tilde{w}^{\text{chg}}_{i,t}=\frac{1/\max(\kappa^{\text{chg}}_{i,t},\varepsilon)}{\sum_j 1/\max(\kappa^{\text{chg}}_{j,t},\varepsilon)}
$$

Target charge energy for each storage:
$$
E^{\text{chg,tar}}_{i,t}=\tilde{w}^{\text{chg}}_{i,t}\;|E^{\text{net}}_t|
$$

**Step 4:** Feasibility check
- Apply constraints (SoC ceiling, charge rate limits)
- Adjust to feasible levels: $E^{\text{chg,act}}_{i,t}$
- Reassign residual to next lowest-cost storage

**Step 5:** Update state of charge
$$
\tilde{s}_{i,t+1}=\mathrm{clamp}\!\left(\tilde{s}_{i,t}+\frac{E^{\text{chg,act}}_{i,t}\,\eta_i^{\uparrow}}{S_i},\,0,\,1\right)
$$

---

## 5) Performance Metrics and Logging

### 5.1 Per-PPU Cost Logging

At each timestep $t$, record the following for every PPU:
$$
d^{\text{stor}}_{i,t},\quad u^{\text{dis}}_{i,t},\quad u^{\text{chg}}_{i,t},\quad c_{i,t},\quad
\kappa^{\text{dis}}_{i,t},\quad \kappa^{\text{chg}}_{i,t},\quad
\tilde{w}^{\text{dis}}_{i,t},\quad \tilde{w}^{\text{chg}}_{i,t}
$$

These metrics enable:
- Post-hoc analysis of dispatch decisions
- Validation of cost-based allocation
- Debugging and optimization

### 5.2 Herfindahl-Hirschman Index (HHI) by PPU Type

The HHI measures market concentration, with higher values indicating less diversity.

**Energy Aggregation**

Choose a reporting period $\mathcal{B}$ (15-min / hourly / daily / monthly).

For each PPU type $k$, compute total energy:

**Production Energy:**
$$
E^{\text{prod}}_{k}=\sum_{t\in\mathcal{B}}\sum_{i\in k} P^{\text{prod}}_{i,t}\,\Delta t
$$

**Storage Discharge Energy:**
$$
E^{\text{stor}}_{k}=\sum_{t\in\mathcal{B}}\sum_{i\in k} P^{\text{dis}}_{i,t}\,\Delta t
$$

**Market Shares**

For $X\in\{\text{prod},\text{stor}\}$:
$$
s^{X}_k=\frac{E^{X}_k}{\sum_{k'}E^{X}_{k'}}
$$

**HHI Calculation**
$$
\mathrm{HHI}^{X}=\sum_k (s^{X}_k)^2 \in \left[\tfrac{1}{K},\,1\right]
$$

where $K$ is the number of PPU types.

**Interpretation:**
- $\mathrm{HHI} \approx 1/K$: Perfect diversity (all types contribute equally)
- $\mathrm{HHI} \approx 1$: High concentration (one type dominates)

---

## 6) Required Inputs

### Time Series Data
- $\mathrm{demand}_t$: System demand (MW) at each timestep
- $\mathrm{nonflex\_supply}_t$: Non-flexible supply (MW) at each timestep
- $\pi_t$: Electricity price or scarcity signal (€/MWh)

### Co-state Values (or Proxies)
- $\lambda^{(H)}_{i,t}$ for $H\in\{1\text{d},3\text{d},7\text{d},30\text{d}\}$
- **If unavailable:** Use proxy $\lambda^{(H)}_{i,t}\approx \eta_i^{\downarrow}\,\mathbb{E}_t[\pi_{t+H}]$

### PPU Technical Parameters
- Storage capacity: $S_i$ (MWh)
- Efficiencies: $\eta_i^{\uparrow}$ (charge), $\eta_i^{\downarrow}$ (discharge)
- All operational limits (externally enforced)

### System Configuration
- PPU type mapping: $i\mapsto k$ (for HHI calculation)
- Scaling parameters: $\alpha_d$, $\alpha_u$, $\alpha_c$ (if using $\tanh$ squashing)
- Target SoC: $\tilde{s}^{\star}_i=0.60$, deadband $\delta_i=0.05$
- Initial SoC: $\tilde{s}_{i,0}=0.60$

### Renewable Collection
- $E^{\text{col}}_t$: Collected renewable energy (MWh) at each timestep
- $\eta^{\text{col}}_t$: Collection efficiency at each timestep

---

## 7) Co-state Intuition: $\lambda^{(H)}_{i,t}$

### Mathematical Definition

In finite-horizon dynamic programming with state $s_{i,t}$ and value function $V^{(H)}_t(s)$:
$$
\lambda^{(H)}_{i,t} \;=\; \frac{\partial V^{(H)}_t}{\partial s_{i,t}}\quad [\text{€/MWh}]
$$

This represents the **marginal value of stored energy** — how much system value increases with one additional MWh in storage.

### Operational Interpretation

Compare current price $\pi_t$ to co-state $\lambda^{(H)}_{i,t}$:

- **$\pi_t \gg \lambda^{(H)}_{i,t}$:** Current price is high relative to future value → **discharge now is attractive**
- **$\pi_t \ll \lambda^{(H)}_{i,t}$:** Future value exceeds current price → **save energy for later**
- **$\pi_t \approx \lambda^{(H)}_{i,t}$:** Neutral (indifferent between now and later)

### Multi-Horizon Averaging

By averaging across multiple horizons (1d, 3d, 7d, 30d), the system balances:
- **Short-term** price volatility (1-day horizon)
- **Medium-term** weather patterns (3-7 day horizons)
- **Long-term** seasonal trends (30-day horizon)

This prevents myopic decisions that optimize for immediate profit while compromising longer-term system resilience.